In [1]:
import qiskit.tools.jupyter
%qiskit_version_table

In [19]:
from qiskit import IBMQ

# IBMQ.save_account(TOKEN)
IBMQ.load_account() # Load account from disk
IBMQ.providers()    # List all available providers

[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>]

### Getting started with OpenPulse

- Build and execute a very basic Pulse program, which is called a schedule.

#### Initialize

In [20]:
# importing the Schedule class from the qiskit.pulse module
# initialize a Schedule with the name "getting_started"

from qiskit.pulse import Schedule

sched = Schedule(name="getting_started") # empty schedule

#### Build instructions
* Add some instructions to our schedule.

`SamplePulse` is the most central instruction for building schedules, It is a pulse signal specified as an array of time-ordered complex amplitudes(samples). Each sample is played for one cycle, a timestep `dt` determined by the backend.
If we want to know the real-time dynamics of our program, we need to know the value of `dt`.

In [21]:
from qiskit.pulse import SamplePulse

my_pulse = SamplePulse([0.00043, 0.0007 , 0.00112, 0.00175, 0.00272, 0.00414, 0.00622,
                        0.00919, 0.01337, 0.01916, 0.02702, 0.03751, 0.05127, 0.06899,
                        0.09139, 0.1192 , 0.15306, 0.19348, 0.24079, 0.29502, 0.35587,
                        0.4226 , 0.49407, 0.56867, 0.64439, 0.71887, 0.78952, 0.85368,
                        0.90873, 0.95234, 0.98258, 0.99805, 0.99805, 0.98258, 0.95234,
                        0.90873, 0.85368, 0.78952, 0.71887, 0.64439, 0.56867, 0.49407,
                        0.4226 , 0.35587, 0.29502, 0.24079, 0.19348, 0.15306, 0.1192 ,
                        0.09139, 0.06899, 0.05127, 0.03751, 0.02702, 0.01916, 0.01337,
                        0.00919, 0.00622, 0.00414, 0.00272, 0.00175, 0.00112, 0.0007 ,
                        0.00043],
                       name="short_gaussian_pulse")

ImportError: cannot import name 'SamplePulse' from 'qiskit.pulse' (C:\Users\muham\anaconda3\envs\qiskit-stable\lib\site-packages\qiskit\pulse\__init__.py)

we've simply passed the exact amplitudes of the pulse envelope we want to play as an array. The array above is a Gaussian function evaluated at 64 points, with an amplitude of 1 and a standard deviation of 8. The (zero-indexed) $i^{th}$ sample will play from time `i*dt` up to `(i + 1)*dt`, modulated by the qubit frequency. Think of this like an arbitrary waveform generator (AWG), playing the samples you give to the `SamplePulse`, mixed with a continuous sine wave generator outputting a tone at the qubit frequency.

The values above happen to be real, but they can also be complex. The amplitude norm of any pulse signal is arbitrarily limited to 1. Each backend system may also impose further constraints -- for instance, a minimum pulse size of 64.

#### Schedule instructions
Next, we have to add an instruction to execute the pulse signal we just built. This means specifying not only the time that the pulse should be played, but also where it should be played. When we build circuits, we specify which qubit a gate operation should be applied to. In Pulse, every qubit has multiple channels.

We will play our pulse on the drive channel of qubit 0. The drive channel lets us enact single qubit operations.

In [15]:
from qiskit.pulse import Play, DriveChannel

qubit_index = 0
sched = sched.insert(0, Play(my_pulse, DriveChannel(qubit_index)))

NameError: name 'my_pulse' is not defined

Note that the pulse we defined operates on the `DriveChannel`, which in turn is initialized with the qubit index. We use `Schedule.insert` to play the pulse at timestep `t = 0`. 

Let's review what we've done, using `Schedule.draw`: 

In [17]:
sched.draw(label=True, scaling=0.5)

TypeError: draw() got an unexpected keyword argument 'label'

This pulse will drive qubit 0. It is modulated at qubit 0's resonant frequency, so it will drive the $|0\rangle$ to $|1\rangle$ transition. It is not calibrated to stop at a particular state, so we won't know what state we've prepared until we look at the results. For our purposes, we don't mind what state we end up in.

All that's left to do is to add a measurement. There is a convenient utility function for adding measurements, but it requires data from the backend system that the program will be running on. We will also need the backend to execute the program.

#### Grab a backend   

In [22]:
from qiskit.test.mock import FakeAlmaden

backend = FakeAlmaden()

#### Add measurements

Now we can use this backend to add the measurement instructions for us.

In [23]:
from qiskit.scheduler import measure_all

sched = sched.insert(sched.duration, measure_all(backend))

C:\Users\muham\AppData\Local\Temp\ipykernel_9104\1203883437.py:3: DeprecationWarning: "measure_all" has been moved to "qiskit.pulse.macros"
  sched = sched.insert(sched.duration, measure_all(backend))


Let's see what the convenience function has added for us, using draw again. The acquisition and measurement pulses are very long compared to our initial pulse, so we can use the `plot_range` argument to clip the schedule.